In [ ]:
import locale

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install torchaudio

In [ ]:
import whisper
import torchaudio
import torch

In [ ]:
def transcribe_audio(audio_file_path, model_name="base.en", device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Transcribes an audio file using the Whisper model.

    Args:
        audio_file_path (str): Path to the audio file.
        model_name (str, optional): Name of the Whisper model to use. Defaults to "base.en".
        device (str, optional): Device to run the model on ("cuda" or "cpu"). Defaults to "cuda" if available, else "cpu".

    Returns:
        str: The transcribed text.
    """

    # Load the audio file
    audio, sample_rate = torchaudio.load(audio_file_path)

    # Check if audio has more than one channel and convert to mono if needed
    if audio.shape[0] > 1:
        audio = torch.mean(audio, dim=0, keepdim=True)

    # Pad/trim and move to device
    audio = whisper.pad_or_trim(audio.flatten()).to(device)
    mel = whisper.log_mel_spectrogram(audio)

    # Load the Whisper model
    model = whisper.load_model(model_name, device=device)

    # Set decoding options
    options = whisper.DecodingOptions(language="en", without_timestamps=True)

    # Decode the audio
    result = model.decode(mel, options)

    return result["text"]

In [ ]:
audio_file_path = "WTFB2.mp4"  # Replace with your audio file path
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
transcript = transcribe_audio(audio_file_path, device=DEVICE)
print(transcript)

# Save the transcript to a file
with open("output.txt", "w") as f:
    f.write(transcript)